In [5]:

from PIL import Image
import numpy as np
import math
from PIL import ImageFont, ImageDraw
import cv2

In [4]:
"""
生成 武汉理工大学数学建模 的水印图片
"""
def create_watermark(text, font_path, font_size=26, opacity=100):
    # Calculate lines
    n = int(math.sqrt(len(text))) + 1
    lines = [text[i:i + n] for i in range(0, len(text), n)]
    # Create a blank image with white background
    width, height = n * font_size, n * font_size
    img = Image.new('RGBA', (width, height), (255, 255, 255))
    # Load font
    font = ImageFont.truetype(font_path, font_size)
    # Initialize ImageDraw
    draw = ImageDraw.Draw(img)
    # Set text color
    text_color = (0, 0, 0, opacity)
    # Draw text on image
    for i, line in enumerate(lines):
        # Calculate the width of the line
        text_bbox = draw.textbbox((0, 0), line, font)
        line_width = text_bbox[2] - text_bbox[0]
        # Calculate the x coordinate to center the line
        x = (width - line_width) / 2
        draw.text((x, i * font_size), line, font=font, fill=text_color)
    # Save the image
    img.save('watermark.png', 'PNG')
# 生成带有武汉理工大学建模的水印图片,保存在桌面
# create_watermark('武汉理工大学数学建模', 'C:/Windows/Fonts/simhei.ttf', font_size=26, opacity=100)


In [22]:
"""
进行DCT变换并嵌入水印
"""
# 定义一维dct变换函数
def dct(signal, axis=0):
    N = signal.shape[axis]
    dct_result = np.zeros_like(signal)

    for k in range(N):
        sum_value = 0
        for n in range(N):
            idx = [slice(None)] * signal.ndim  # 创建切片索引
            idx[axis] = n
            sum_value += signal[tuple(idx)] * np.cos((np.pi / N) * (n + 0.5) * k)
        if k == 0:
            dct_result[tuple(idx)] = sum_value / np.sqrt(N)
        else:
            dct_result[tuple(idx)] = sum_value * np.sqrt(2/N)

    return dct_result
# 对原始图像执行离散余弦变换（DCT）
def perform_dct(original_array):
    height, width, _ = original_array.shape
    dct_blocks = np.empty_like(original_array, dtype=np.float64)
    # 对dct_blocks的整体进行dct
    dct_blocks = cv2.dct(original_array.astype(np.float64))
    return dct_blocks


"""
定义图像读取函数
"""
def load_image(image_path):
    return Image.open(image_path)
"""
定义图像转换数组函数
"""
def image_to_array(image):
    return np.asarray(image, dtype=np.float64)
# 定义数组转换图像函数
def array_to_image(array):
    array = array.clip(0, 255)
    array = array.astype('uint8')
    return Image.fromarray(array)
# 定义提取水印函数
def extract_watermark(dct_blocks_with_watermark, original_dct_blocks, alpha=0.05):
    watermark_blocks = (dct_blocks_with_watermark - original_dct_blocks) / alpha
    watermark_array = np.zeros_like(original_dct_blocks, dtype=np.float64)
    watermark_array[::8, ::8] = watermark_blocks[::8, ::8]
    return watermark_array
# 定义裁剪和转换函数
def clip_and_convert(watermark_array):
    watermark_array = watermark_array.clip(-255, 255)
    watermark_array = watermark_array.astype('int16')
    watermark_array[::8, ::8] = watermark_array[::8, ::8] / 100
    return watermark_array

# 将水印嵌入到DCT块中
def embed_watermark(dct_blocks, watermark_array, alpha=0.05):
    dct_blocks_with_watermark = dct_blocks.copy()
    dct_blocks_with_watermark[::8, ::8] += alpha * watermark_array
    return dct_blocks_with_watermark
# 将水印嵌入后的DCT块转换回图像
def perform_idct(dct_blocks_with_watermark):
    height, width, _ = dct_blocks_with_watermark.shape
    idct_blocks_with_watermark = np.empty_like(dct_blocks_with_watermark, dtype=np.float64)
    # 对dct_blocks_with_watermark的整体进行idct
    idct_blocks_with_watermark = cv2.idct(dct_blocks_with_watermark.astype(np.float64))
    return idct_blocks_with_watermark

    return idct_blocks_with_watermark
def process_images(image_with_watermark_path, original_image_path, alpha=0.05):
    # 加载图像
    image_with_watermark = load_image(image_with_watermark_path)
    original_image = load_image(original_image_path)
    # 将图像转换为数组
    image_with_watermark_array = image_to_array(image_with_watermark)
    original_array = image_to_array(original_image)
    # 对图像执行DCT
    dct_blocks_with_watermark = perform_dct(image_with_watermark_array)
    original_dct_blocks = perform_dct(original_array)
    # 提取水印
    watermark_array = extract_watermark(dct_blocks_with_watermark, original_dct_blocks, alpha)
    # 裁剪和转换图像
    watermark_array = clip_and_convert(watermark_array)
    # 将数组转换回图像
    watermark_image = array_to_image(watermark_array)
    return watermark_image
"""
将水印嵌入到图像中
"""
def embed_watermark_in_image(image_path, watermark_path, alpha=0.05):
    # 加载图像
    image = load_image(image_path)
    watermark = load_image(watermark_path)
    # 将图像转换为数组
    image_array = image_to_array(image)
    watermark_array = image_to_array(watermark)
    # 对图像执行DCT
    dct_blocks = perform_dct(image_array)
    # 将水印嵌入到DCT块中
    dct_blocks_with_watermark = embed_watermark(dct_blocks, watermark_array, alpha)
    # 将DCT块转换回图像
    image_with_watermark_array = perform_idct(dct_blocks_with_watermark)
    # 将数组转换回图像
    image_with_watermark = array_to_image(image_with_watermark_array)
    return image_with_watermark

In [23]:
"""
水印是watermark.png,原图片是raw_pic.jpg
"""
# 将水印嵌入到图像中
image_with_watermark = embed_watermark_in_image('raw_pic.jpg', 'watermark.png')

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\dxt.cpp:4417: error: (-215:Assertion failed) type == CV_32FC1 || type == CV_64FC1 in function 'cv::dct'
